# A Distributed Matrix Data Structure and Its Statistical Applications on PyTorch

**NOTE: This notebook is incomplete. At this time, it is uploaded for testing purposes, in preparation of tutorials to be presented in the programming workshop at the inaugural Lange's Symposium on Feb 21-22, 2020.**

## Synopsis

We developed a distributed matrix operation package suitable for distributed matrix-vector operations and distributed tall-and-thin (or wide-and-short) matrices.
The code runs on both multi-node machines and multi-GPU machines using PyTorch.
We apply this package for four statistical applications, namely, nonnegative matrix factorization (NMF), multidimensional scaling (MDS), positron emission tomography (PET), and $\ell_1$-regularized Cox regression.
In particular, $\ell_1$-regularized Cox regression with the UK Biobank dataset was the biggest multivariate survival analysis to our knowledge. 
In this workshop, we provide small examples that run on a single node, and demonstrate multi-GPU usage on our own machine.

## Introduction to PyTorch

What is PyTorch?

## Basic PyTorch Operations

We introduce simple operations on PyTorch. Note that Python uses 0-based, rowmajor ordering, like C and C++ (R is 1-based, column-major ordering). First we import the PyTorch
library. This is equvalent to library() in R.

In [1]:
import torch

In [2]:
torch.__version__

'1.1.0'

### Tensor Creation

One may create an uninitialized tensor. This creates a 3 × 4 tensor (matrix).

In [3]:
torch.empty(3, 4) # uninitialized tensor

tensor([[5.9179e+09, 4.5741e-41, 5.9179e+09, 4.5741e-41],
        [1.9421e+31, 2.7491e+20, 6.1949e-04, 1.2140e+33],
        [3.2307e-18, 7.1760e+22, 7.2250e+28, 2.5226e-18]])

The following is equivalent to `set.seed()` in R.

In [4]:
torch.manual_seed(100)

This generates a tensor initialized with random values from (0, 1).

In [5]:
y = torch.rand(3, 4) # from Unif(0, 1)
y

tensor([[0.1117, 0.8158, 0.2626, 0.4839],
        [0.6765, 0.7539, 0.2627, 0.0428],
        [0.2080, 0.1180, 0.1217, 0.7356]])

We can also generate a tensor filled with zeros or ones.

In [6]:
z = torch.ones(3, 4) # torch.zeros(3, 4)
z

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])

A tensor can be created from standard Python data.

In [7]:
w = torch.tensor([3, 4, 5, 6])
w

tensor([3, 4, 5, 6])

A tensor can be created in certain datatype (default: float32) and on certain device (default: CPU) of choice 

In [8]:
# double precision
w = torch.tensor([3, 4, 5, 6], dtype=torch.float64)
w

tensor([3., 4., 5., 6.], dtype=torch.float64)

In [9]:
# on GPU number zero. will not run if CUDA GPU is not present.
w = torch.tensor([3, 4, 5, 6], device='cuda:0')
w

AssertionError: 
Found no NVIDIA driver on your system. Please check that you
have an NVIDIA GPU and installed a driver from
http://www.nvidia.com/Download/index.aspx

Shape of a tensor can be accessed by appending `.shape` to the tensor name.

In [10]:
z.shape

torch.Size([3, 4])

### Casting

A tensor can change datatype and location by the method `.to()`. The arguments are similar to choosing datatype and device of the new tensor.

In [11]:
w = w.to(device = "cpu", dtype=torch.int32)
w

tensor([3, 4, 5, 6], dtype=torch.int32)

### Indexing

The following are standard method of indexing tensors.

In [12]:
y[2, 3] # indexing: zero-based, returns a 0-dimensional tensor

tensor(0.7356)

The indexing always returns a (sub)tensor, even for scalars (treated as zero-dimensional tensors).
A standard Python number can be returned by using .item().

In [13]:
y[2, 3].item() # A standard Python floating-point number

0.7355988621711731

To get a column from a tensor, we use the indexing as below. The syntax is similar but slightly
different from R.

In [14]:
y[:, 3] # 3rd column. The leftmost column is 0th. cf. y[, 4] in R

tensor([0.4839, 0.0428, 0.7356])

The following is for taking a row.

In [15]:
y[2, :] # 2nd row. The top row is 0th. cf. y[3, ] in R

tensor([0.2080, 0.1180, 0.1217, 0.7356])

### Simple operations

Here we provide an example of simple operations on PyTorch. Addition using the operator ‘+’ acts
just like anyone can expect:

In [16]:
x = y + z # a simple addition.
x

tensor([[1.1117, 1.8158, 1.2626, 1.4839],
        [1.6765, 1.7539, 1.2627, 1.0428],
        [1.2080, 1.1180, 1.1217, 1.7356]])

Here is another form of addition.

In [17]:
x = torch.add(y, z) # another syntax for addition

The operators ending with an underscore (`_`) changes the value of the tensor in-place. Otherwise, the argument never changes. Unlike methods ending with `!` in Julia, this rule is strictly enforced in PyTorch. (The underscore determines usage of the keyword `const` in C++-level.)

In [18]:
y.add_(z) # in-place addition

tensor([[1.1117, 1.8158, 1.2626, 1.4839],
        [1.6765, 1.7539, 1.2627, 1.0428],
        [1.2080, 1.1180, 1.1217, 1.7356]])

### Concatenation

We can concatenate the tensors using the function `cat()`, which resembles `c()`, `cbind()`, and
`rbind()` in R. The second argument indicates the dimension that the tesors are concatenated
along: zero means by concatenation by rows, and one means by columns.

In [19]:
torch.cat((y, z), 0) # along the rows

tensor([[1.1117, 1.8158, 1.2626, 1.4839],
        [1.6765, 1.7539, 1.2627, 1.0428],
        [1.2080, 1.1180, 1.1217, 1.7356],
        [1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000]])

In [20]:
torch.cat((y, z), 1) # along the columns

tensor([[1.1117, 1.8158, 1.2626, 1.4839, 1.0000, 1.0000, 1.0000, 1.0000],
        [1.6765, 1.7539, 1.2627, 1.0428, 1.0000, 1.0000, 1.0000, 1.0000],
        [1.2080, 1.1180, 1.1217, 1.7356, 1.0000, 1.0000, 1.0000, 1.0000]])

### Reshaping

One can reshape a tensor, like changing the attribute `dim` in R.

In [21]:
y.view(12) # 1-dimensional array

tensor([1.1117, 1.8158, 1.2626, 1.4839, 1.6765, 1.7539, 1.2627, 1.0428, 1.2080,
        1.1180, 1.1217, 1.7356])

Up to one of the arguments of `view()` can be −1. The size of the reshaped tensor is inferred
from the other dimensions.

In [22]:
# reshape into (6)-by-2 tensor;
# (6) is inferred from the other dimension
y.view(-1, 2)

tensor([[1.1117, 1.8158],
        [1.2626, 1.4839],
        [1.6765, 1.7539],
        [1.2627, 1.0428],
        [1.2080, 1.1180],
        [1.1217, 1.7356]])

### Basic statistics

Calling `.sum()`, `.mean()`, `.std()` methods of a tensor do the obvious. Optional argument determines the dimension of reduction.

In [23]:
y

tensor([[1.1117, 1.8158, 1.2626, 1.4839],
        [1.6765, 1.7539, 1.2627, 1.0428],
        [1.2080, 1.1180, 1.1217, 1.7356]])

In [24]:
y.sum()

tensor(16.5933)

In [25]:
y.sum(0) # reduces rows, columnwise sum

tensor([3.9962, 4.6878, 3.6469, 4.2623])

In [26]:
y.sum(1) # reduces columns, rowwise sum

tensor([5.6739, 5.7359, 5.1834])

In [27]:
y.sum((0, 1)) # reduces rows and columns -> a single number.

tensor(16.5933)

In [28]:
y.mean()

tensor(1.3828)

In [29]:
y.mean(0)

tensor([1.3321, 1.5626, 1.2156, 1.4208])

In [30]:
y.std(1)

tensor([0.3058, 0.3384, 0.2961])

### Linear Algebra

Matrix transpose is performed by appending `.t()` to a tensor. Matrix multiplication is carried out by the method `torch.mm()`.

In [31]:
torch.mm(y, z.t())

tensor([[5.6739, 5.6739, 5.6739],
        [5.7359, 5.7359, 5.7359],
        [5.1834, 5.1834, 5.1834]])

## `torch.distributed`: Distributed subpackage for PyTorch

`torch.distributed` is the subpackage for distributed operations on PyTorch. The interface is mostly inspired by the message passing interface (MPI). The available backends are:

* Gloo, a collective communication library developed by Facebook, included in PyTorch. Full support for CPU, partial collective communication only for GPU.
* MPI, a good-old communication standard. The most flexible, but PyTorch needs to be compiled from its source to use it as a backend. Full support for GPU if the MPI installation is "CUDA-aware".
* NCCL, Nvidia Collective Communications Library, collective communication only for multiple GPUs on the same machine.

For this workshop, we use Gloo for its full functionalities on CPU and runnability on Jupyter Notebook. The experiments in our paper use MPI for running multi-node setting and multi-GPU setting with basically the same code. The interface below is specific for Gloo backend. For MPI backend, please consult with a section from [distributed package tutorial](https://pytorch.org/tutorials/intermediate/dist_tuto.html#communication-backends) or [our code](https://github.com/kose-y/dist_stat/tree/master/examples).

In [32]:
import os
import torch
import torch.distributed as dist
from torch.multiprocessing import Process

def init_process(rank, size, fn, backend='gloo'):
    """ Initialize the distributed environment. """
    os.environ['MASTER_ADDR'] = '127.0.0.1'
    os.environ['MASTER_PORT'] = '29500'
    dist.init_process_group(backend, rank=rank, world_size=size)
    fn(rank, size)

def run_process(size, fn):
    processes = []
    for rank in range(size):
        p = Process(target=init_process, args=(rank, size, fn))
        p.start()
        processes.append(p)
        
    for p in processes:
        p.join()

### Point-to-point communication

![](https://pytorch.org/tutorials/_images/send_recv.png)
Figure courtesy of: https://pytorch.org/tutorials/_images/send_recv.png

In [33]:
"""Blocking point-to-point communication."""

def point_to_point(rank, size):
    tensor = torch.zeros(1)
    if rank == 0:
        tensor += 1
        # Send the tensor to process 1
        dist.send(tensor=tensor, dst=1)
    elif rank == 1:
        # Receive tensor from process 0
        dist.recv(tensor=tensor, src=0)
    dist.barrier()
    print('Rank ', rank, ' has data ', tensor[0])

In [34]:
run_process(4, point_to_point)

Rank  1  has data  tensor(1.)
Rank  3  has data  tensor(0.)
Rank  2  has data  tensor(0.)
Rank  0  has data  tensor(1.)


### Collective communication

| | | 
|:---|:---|
| ![](https://pytorch.org/tutorials/_images/scatter.png) | ![](https://pytorch.org/tutorials/_images/gather.png) |
| Scatter | Gather |
| ![](https://pytorch.org/tutorials/_images/reduce.png) | ![](https://pytorch.org/tutorials/_images/all_reduce.png) |
| Reduce | All-reduce |
| ![](https://pytorch.org/tutorials/_images/broadcast.png) | ![](https://pytorch.org/tutorials/_images/all_gather.png) |
| Broadcast | All-gather |

Table courtesy of: https://pytorch.org/tutorials/intermediate/dist_tuto.html#communication-backends


In [35]:
def broadcast(rank, size):
    tensor = torch.zeros(1)
    if rank == 0:
        tensor[0] = 7
    dist.broadcast(tensor, src=0)
    print('Rank ', rank, ' has data ', tensor[0])

In [36]:
run_process(4, broadcast)

Rank  2  has data  tensor(7.)
Rank  3  has data  tensor(7.)
Rank  1  has data  tensor(7.)
Rank  0  has data  tensor(7.)


In [37]:
def reduce(rank, size):
    tensor = torch.ones(1)
    dist.reduce(tensor, 3)
    print('Rank ', rank, ' has data ', tensor[0])

In [38]:
run_process(4, reduce)

Rank  2  has data  tensor(2.)
Rank  3  has data  tensor(4.)
Rank  1  has data  tensor(3.)
Rank  0  has data  tensor(4.)


In [39]:
def all_reduce(rank, size):
    tensor = torch.ones(1)
    dist.all_reduce(tensor, op=dist.ReduceOp.SUM)
    print('Rank ', rank, ' has data ', tensor[0])

In [40]:
run_process(4, all_reduce)

Rank  3  has data  tensor(4.)
Rank  1  has data  tensor(4.)
Rank  2  has data  tensor(4.)
Rank  0  has data  tensor(4.)


In [41]:
def all_gather(rank, size):
    tensors = [torch.zeros(1) for i in range(size)]
    dat = torch.zeros(1)
    dat[0] += rank
    dist.all_gather(tensors, dat)
    print('Rank ', rank, ' has data ', tensors)

In [42]:
run_process(4, all_gather)

Rank  1  has data  [tensor([0.]), tensor([1.]), tensor([2.]), tensor([3.])]
Rank  2  has data  [tensor([0.]), tensor([1.]), tensor([2.]), tensor([3.])]
Rank  3  has data  [tensor([0.]), tensor([1.]), tensor([2.]), tensor([3.])]
Rank  0  has data  [tensor([0.]), tensor([1.]), tensor([2.]), tensor([3.])]


## `distmat`: Distributed Matrices on PyTorch

Using the tensor operations and communication package, we created a data structure for a distributed matrix. In this structure, each process,
enumerated by its rank, holds a contiguous block of the full data matrix by rows or columns.
The data may be a sparse matrix. If GPUs are involved, each process controls a GPU whose
index matches the process rank. For notational simplicity, we denote the dimension to split
in square brackets. If a [100] × 100 matrix is split over four processes, the process with rank
0 keeps the first 25 rows of the matrix, and the rank 3 process takes the last 25 rows. For
the sake of simplicity, we always assume that the size along the split dimension is divided
by the number of processes.

In [43]:
import dist_stat.distmat as distmat

### Creation

- `distgen_ones()`: Creates a distributed matrix filled with ones
- `distgen_zeros()`: Creates a distributed matrix filled with zeros
- `distgen_uniform()`: Creates a distributed matrix from uniform distribution
- `distgen_normal()`: Creates a distributed matrix from stndard normal distribution 

In [59]:
def create_unif(rank, size):
    A = distmat.distgen_uniform(8, 8, TType=torch.DoubleTensor)
    print('Matrix A:', 'Rank ', rank, ' has data ', A.chunk)   

In [72]:
run_process(4, create_unif)

Matrix A: Rank  1  has data  tensor([[0.5459, 0.3573, 0.2406, 0.5600, 0.6501, 0.9692, 0.5168, 0.2422],
        [0.4312, 0.5917, 0.3425, 0.2202, 0.7030, 0.5629, 0.9259, 0.7612]],
       dtype=torch.float64)
Matrix A: Rank  3  has data  tensor([[0.6214, 0.7096, 0.7099, 0.7977, 0.8779, 0.1236, 0.5650, 0.0655],
        [0.1210, 0.0454, 0.5070, 0.1860, 0.5035, 0.1454, 0.8090, 0.4991]],
       dtype=torch.float64)
Matrix A: Rank  2  has data  tensor([[0.8381, 0.4943, 0.5984, 0.6167, 0.6128, 0.8593, 0.1344, 0.5146],
        [0.1479, 0.4238, 0.5144, 0.7051, 0.8133, 0.1795, 0.2721, 0.4631]],
       dtype=torch.float64)
Matrix A: Rank  0  has data  tensor([[0.6941, 0.3464, 0.9751, 0.7911, 0.4274, 0.4460, 0.5522, 0.9559],
        [0.9405, 0.2215, 0.3271, 0.1352, 0.6283, 0.3030, 0.1302, 0.1811]],
       dtype=torch.float64)


In [81]:
def from_chunks(rank, size):
    torch.manual_seed(100 + rank)
    chunk = torch.randn(2, 4)
    print("rank ", rank, "has chunk", chunk)
    A = distmat.THDistMat.from_chunks(chunk)
    print('Matrix A:', 'Rank ', rank, ' has data ', A.chunk)       

In [82]:
run_process(4, from_chunks)

rank  2 has chunk tensor([[ 0.9907,  0.3349,  1.1497, -0.5498],
        [-0.1046,  2.0104, -0.7886, -0.1246]])rank  1 has chunk tensor([[-1.3905, -0.8152, -0.3204,  0.7377],
        [-1.7534,  0.6033, -0.2520, -0.4373]])rank  0 has chunk tensor([[ 0.3607, -0.2859, -0.3938,  0.2429],
        [-1.3833, -2.3134, -0.3172, -0.8660]])
rank  3 has chunk tensor([[ 1.7286, -0.4007,  2.5587,  1.6848],
        [-1.6571, -0.2811,  0.7743, -0.9554]])


Matrix A: Rank  1  has data  tensor([[-1.3905, -0.8152, -0.3204,  0.7377],
        [-1.7534,  0.6033, -0.2520, -0.4373]])Matrix A: Rank  3  has data  tensor([[ 1.7286, -0.4007,  2.5587,  1.6848],
        [-1.6571, -0.2811,  0.7743, -0.9554]])
Matrix A: Rank  2  has data  tensor([[ 0.9907,  0.3349,  1.1497, -0.5498],
        [-0.1046,  2.0104, -0.7886, -0.1246]])
Matrix A: Rank  0  has data  tensor([[ 0.3607, -0.2859, -0.3938,  0.2429],
        [-1.3833, -2.3134, -0.3172, -0.8660]])



### Elementwise operations

Some of the basic functions work naturally. 

In [ ]:
C = A + B

In [ ]:
A.log()

For general functions, we have `.apply()`, `.apply_binary()`, `.apply_inplace()`, and `.apply_inplace_binary()`.

### Reductions (sum, product, max, min)

In [70]:
def test_distmat(rank, size):
    
    if rank==0:
        p = 12; q = 12
        data = torch.DoubleTensor(p,q).normal_()
        print(data)
    else:
        data = None

    rslt = distmat.dist_data(data, src=0, TType=torch.DoubleTensor)

    print(rank, rslt.chunk)

    r1 = rslt.diag()

    print(r1.chunk)

    r2 = rslt.diag(distribute=False)
    print(r2)

    rr = rslt + rslt

    print(rank, rslt.chunk, rr.chunk)

    dist.barrier()

    rslt += rslt
    print("rslt += rslt", rank, rslt.chunk)

    print((1 + rslt).chunk)
    print((rslt + torch.arange(12, out=torch.DoubleTensor(1,12))).chunk)
    # print((torch.arange(12, out=TType(1,12)) + rslt).chunk) # doesn't work

    if rank==0:
        col0 = data[:, 0].view(-1, 1)
    else:
        col0 = None
    if rank==0:
        print(col0)
    col0_dist = distmat.dist_data(col0, src=0, TType=torch.DoubleTensor)
    print("adding by 1st col of data", rank, (rslt + col0_dist).chunk)
    rslt.fill_diag_(0)
    print("rslt, diagonal set to zero", rank, rslt.chunk)

In [71]:
run_process(4, test_distmat)

tensor([[ 1.4328e+00,  1.6422e-01, -1.2654e+00,  1.1688e+00, -7.3106e-01,
         -3.5541e-01,  8.6673e-01,  1.0486e+00, -5.6238e-01,  9.0757e-01,
          2.4049e+00,  1.3290e+00],
        [-7.6193e-01,  1.0270e+00,  9.2496e-01,  2.2676e+00, -1.1411e+00,
         -7.8844e-01, -4.0719e-01,  2.3872e-01, -4.2209e-01, -2.4347e+00,
          1.0778e+00,  5.2347e-02],
        [ 5.2616e-01, -5.1227e-01,  6.2022e-01,  1.2590e+00, -1.3863e+00,
         -1.0154e+00, -5.4141e-01, -7.4287e-01,  1.1423e+00, -1.0365e+00,
         -1.3453e+00, -3.8520e-01],
        [ 5.3353e-01,  8.4845e-01, -7.4295e-02, -1.1702e+00,  1.5286e+00,
          5.3773e-01, -1.2187e-01, -1.3303e+00, -1.2701e+00,  1.7897e+00,
          5.3221e-01,  2.7608e-01],
        [ 1.0102e+00,  1.5088e+00, -1.5716e+00,  6.9977e-01, -2.0505e+00,
          3.1391e-01,  4.6768e-01, -3.6801e-01,  9.6022e-01,  4.4294e-01,
         -6.9029e-02,  1.6452e+00],
        [-4.5622e-02,  4.0654e-01, -1.2026e+00,  2.0411e-03, -1.8043e+00,
      

          3.0512e+00, -3.0545e-01]], dtype=torch.float64)

 tensor([[-0.6854,  4.3409, -2.3149,  2.3966,  2.5702, -1.0727,  0.6537,  0.9108,
          2.0153, -4.3491, -0.2257,  2.3507],
        [ 2.0062,  1.5177,  3.8994, -1.2174, -1.5987, -0.2537, -1.1596, -0.3665,
         -2.6495, -3.4704, -1.6493, -1.1081],
        [-4.8354,  1.0587, -2.2134,  1.6090,  2.5504, -3.2844,  2.9482, -0.4993,
          0.9596,  3.6309,  4.5908, -1.7113]], dtype=torch.float64)
rslt += rslt 1 tensor([[ 1.0671e+00,  1.6969e+00, -1.4859e-01, -2.3405e+00,  3.0573e+00,
          1.0755e+00, -2.4375e-01, -2.6605e+00, -2.5401e+00,  3.5793e+00,
          1.0644e+00,  5.5216e-01],
        [ 2.0204e+00,  3.0176e+00, -3.1433e+00,  1.3995e+00, -4.1010e+00,
          6.2783e-01,  9.3536e-01, -7.3602e-01,  1.9204e+00,  8.8589e-01,
         -1.3806e-01,  3.2904e+00],
        [-9.1244e-02,  8.1309e-01, -2.4052e+00,  4.0822e-03, -3.6087e+00,
          1.6516e-01,  1.0075e+00,  1.3882e+00, -4.4174e+00,  2.6927e+00,
      

          0.0000,  3.6309,  4.5908, -1.7113]], dtype=torch.float64)rslt, diagonal set to zero 0 tensor([[ 0.0000,  0.3284, -2.5308,  2.3376, -1.4621, -0.7108,  1.7335,  2.0972,
         -1.1248,  1.8151,  4.8099,  2.6580],
        [-1.5239,  0.0000,  1.8499,  4.5352, -2.2822, -1.5769, -0.8144,  0.4774,
         -0.8442, -4.8694,  2.1556,  0.1047],
        [ 1.0523, -1.0245,  0.0000,  2.5180, -2.7726, -2.0308, -1.0828, -1.4857,
          2.2845, -2.0730, -2.6905, -0.7704]], dtype=torch.float64)

rslt, diagonal set to zero 1 tensor([[ 1.0671e+00,  1.6969e+00, -1.4859e-01,  0.0000e+00,  3.0573e+00,
          1.0755e+00, -2.4375e-01, -2.6605e+00, -2.5401e+00,  3.5793e+00,
          1.0644e+00,  5.5216e-01],
        [ 2.0204e+00,  3.0176e+00, -3.1433e+00,  1.3995e+00,  0.0000e+00,
          6.2783e-01,  9.3536e-01, -7.3602e-01,  1.9204e+00,  8.8589e-01,
         -1.3806e-01,  3.2904e+00],
        [-9.1244e-02,  8.1309e-01, -2.4052e+00,  4.0822e-03, -3.6087e+00,
          0.0000e+00,  1.0075

### Matrix multiplications

Six different scenarios of matrix-matrix multiplications, each representing a different configuration of the split dimension of two input
matrices and the output matrix, were considered and implemented. 

< Content of Table 1 here >



The implementation of each case is carried
out using the collective communication directives. Matrix multiplication scenarios are automatically selected based on the shapes of the input matrices A and
B, except for the Scenarios 1 and 3 sharing the same input structure. Those two are further
distinguished by the shape of output, AB. The nonnegative matrix factorization involves Scenarios 1 to 5.
Scenario 6 is for matrix-vector multiplications, where broadcasting small vectors is almost
always efficient.

In [85]:
from dist_stat.distmm import *
def test_distmm(rank, size):
    TType = torch.DoubleTensor
    p = 12; q = 8; r = 2
    if rank==0:
        fat   = TType(p, q).normal_()
        thin1 = TType(q, r).normal_()
        thin2 = TType(p, r).normal_()

    else:
        fat, thin1, thin2 = None, TType(q,r), TType(p,r)
    dist.broadcast(thin1,0)
    dist.broadcast(thin2,0)

    fat_dist   = distmat.dist_data(fat, src=0, TType=TType)
    thin1_dist = distmat.dist_data(thin1, src=0, TType=TType)
    thin2_dist = distmat.dist_data(thin2, src=0, TType=TType)

    # test distmm_thinthin_inner 
    if rank==0: 
        print("distmm_thinthin_inner: thin1^T x thin1")
        thin1_thin1 = torch.mm(torch.t(thin1), thin1)
        print("thin1^T x thin1: ", thin1_thin1)

    thin1_thin1_bd = distmm_thinthin_inner(thin1_dist.t(), thin1_dist)
    print("rslt in rank %d: "%(rank,), thin1_thin1_bd)

    dist.barrier()


    if rank==0:
        print("distmm_db_d: thin2 x thin1_thin1")
        correct = torch.mm(thin2, thin1_thin1)
        print(correct)
    rslt_dist = distmm_db_d(thin2_dist, thin1_thin1_bd)
    print("rslt in rand %d: "%(rank,), rslt_dist.chunk)
    print(rslt_dist.byrow)


    dist.barrier()

    if rank==0:
        print("distmm_db_d (reverse): thin1_thin1 x thin2^T")
        correct = torch.mm(thin1_thin1, torch.t(thin2))
        print(correct)
    rslt_dist = distmm_db_d(thin2_dist.t(), thin1_thin1_bd, True)
    print("rslt in rand %d: "%(rank,), rslt_dist.chunk)
    print(rslt_dist.byrow)

    dist.barrier()

    if rank==0:
        print("_distmm_fatthin_byrow: fat x thin1")
        correct = torch.mm(fat, thin1)
        print(correct)
    rslt_dist = distmm_fatthin(fat_dist, thin1_dist)
    print("rslt in rand %d: "%(rank,), rslt_dist.chunk)
    print(rslt_dist.byrow)

    dist.barrier()

    if rank==0:
        print("_distmm_fatthin_byrow (reverse): thin1^T x fat^T")
        correct = torch.mm(torch.t(thin1), torch.t(fat))
        print(correct)
    rslt_dist = distmm_fatthin(fat_dist.t(), thin1_dist.t(), reverse=True)
    print("rslt in rand %d: "%(rank,), rslt_dist.chunk)
    print(rslt_dist.byrow)

    dist.barrier()

    if rank==0:
        print("_distmm_thinfat_byrow: thin2^T x fat" )
        # Note: this is reverse for distmm_fatthin, non-reverse for inner ftn
        correct = torch.mm(torch.t(thin2), fat)
        print(correct)
    rslt_dist = distmm_fatthin(fat_dist, thin2_dist.t(), reverse=True, 
                                out_sizes=thin1_dist.sizes)
    print("rslt in rand %d: "%(rank,), rslt_dist.chunk)
    print(rslt_dist.byrow)

    dist.barrier()

    if rank==0:
        print("_distmm_thinfat_byrow (reverse): fat^T x thin2" )
        # Note: this is reverse for distmm_fatthin, non-reverse for inner ftn
        correct = torch.mm(torch.t(fat), thin2)
        print(correct)
    rslt_dist = distmm_fatthin(fat_dist.t(), thin2_dist, reverse=False, 
                                out_sizes=thin1_dist.sizes)
    print("rslt in rand %d: "%(rank,), rslt_dist.chunk)
    print(rslt_dist.byrow)

    if rank==0:
        print("distmm_thinthin_outer: thin1 x thin2^T" )
        correct = torch.mm(thin1, torch.t(thin2))
        print(correct)
    rslt_dist = distmm_thinthin_outer(thin1_dist, thin2_dist.t())
    print("rslt in rank %d: "%(rank,), rslt_dist.chunk)
    print(rslt_dist.byrow)

    if rank==0:
        print("distmm_db_b: thin1^T x thin1(dense)" )
        correct = torch.mm(torch.t(thin1), thin1)
        print(correct)
    rslt_dist = distmm_db_b(thin1_dist.t(), thin1)
    print("rslt in rank %d: "%(rank,), rslt_dist)


    dist.barrier()
    if rank==0:
        print("now we check distributed sparse matrices.")

    def to_sparse(x):
        """ converts dense tensor x to sparse format """
        x_typename = torch.typename(x).split('.')[-1]
        sparse_tensortype = getattr(torch.sparse, x_typename)

        indices = torch.nonzero(x)
        if len(indices.shape) == 0:  # if all elements are zeros
            return sparse_tensortype(*x.shape)
        indices = indices.t()
        values = x[tuple(indices[i] for i in range(indices.shape[0]))]
        return sparse_tensortype(indices, values, x.size())
    
    thin1_sparse_chunk = to_sparse(thin1_dist.chunk)
    thin1_sparse_dist = THDistMat.from_chunks(thin1_sparse_chunk)
    thin2_sparse_chunk = to_sparse(thin2_dist.chunk)
    thin2_sparse_dist = THDistMat.from_chunks(thin2_sparse_chunk)
    print(thin1_sparse_chunk.shape)
    print(thin1_sparse_dist.t().shape)
    print(thin1_dist.shape)
    if rank==0:
        print("correct: ", torch.mm(thin1.t(), thin1))

    r =  distmat.mm(thin1_sparse_dist.t(), thin1_dist )
    print("rslt: ", r)
    

In [86]:
run_process(4, test_distmm)

distmm_thinthin_inner: thin1^T x thin1
thin1^T x thin1:  tensor([[18.8603, -5.5108],
        [-5.5108, 13.0393]], dtype=torch.float64)


/srv/conda/envs/notebook/lib/python3.7/site-packages/torch/distributed/distributed_c10d.py:100: UserWarning: torch.distributed.reduce_op is deprecated, please use torch.distributed.ReduceOp instead
  warnings.warn("torch.distributed.reduce_op is deprecated, please use "
/srv/conda/envs/notebook/lib/python3.7/site-packages/torch/distributed/distributed_c10d.py:100: UserWarning: torch.distributed.reduce_op is deprecated, please use torch.distributed.ReduceOp instead
  warnings.warn("torch.distributed.reduce_op is deprecated, please use "
/srv/conda/envs/notebook/lib/python3.7/site-packages/torch/distributed/distributed_c10d.py:100: UserWarning: torch.distributed.reduce_op is deprecated, please use torch.distributed.ReduceOp instead
  warnings.warn("torch.distributed.reduce_op is deprecated, please use "
/srv/conda/envs/notebook/lib/python3.7/site-packages/torch/distributed/distributed_c10d.py:100: UserWarning: torch.distributed.reduce_op is deprecated, please use torch.distributed.Reduce

rslt in rank 0:  tensor([[18.8603, -5.5108],
        [-5.5108, 13.0393]], dtype=torch.float64)
rslt in rank 1:  tensor([[18.8603, -5.5108],
        [-5.5108, 13.0393]], dtype=torch.float64)rslt in rank 2:  tensor([[18.8603, -5.5108],
        [-5.5108, 13.0393]], dtype=torch.float64)rslt in rank 3:  tensor([[18.8603, -5.5108],
        [-5.5108, 13.0393]], dtype=torch.float64)


distmm_db_d: thin2 x thin1_thin1
tensor([[ 20.4717, -20.9301],
        [ -0.3866,  -2.3102],
        [  3.4683,  -1.9985],
        [ 10.0434, -21.7488],
        [ -8.9008,   1.7786],
        [ 14.3755, -13.4916],
        [ 14.6289,   6.1289],
        [-11.1466,  -3.2908],
        [ -4.6314,  17.0375],
        [ -4.6952,  -2.8811],
        [ 27.1388,  -5.2625],
        [ -7.5063,  -3.7725]], dtype=torch.float64)
rslt in rand 3:  tensor([[-4.6952, -2.8811],
        [27.1388, -5.2625],
        [-7.5063, -3.7725]], dtype=torch.float64)rslt in rand 0:  tensor([[ 20.4717, -20.9301],
        [ -0.3866,  -2.3102],
      

        [-5.5108, 13.0393]], dtype=torch.float64)[8, 2]

rslt:  tensor([[18.8603, -5.5108],
        [-5.5108, 13.0393]], dtype=torch.float64)rslt:  tensor([[18.8603, -5.5108],
        [-5.5108, 13.0393]], dtype=torch.float64)rslt:  tensor([[18.8603, -5.5108],
        [-5.5108, 13.0393]], dtype=torch.float64)


rslt:  tensor([[18.8603, -5.5108],
        [-5.5108, 13.0393]], dtype=torch.float64)


## Nonnegative Matrix Factorization (NMF)

The following code is a simplified version. The full object-oriented version is available at the [GitHub repo](https://github.com/kose-y/dist_stat).

In [ ]:
def nmf(rank, size):
    # TODO: initialization here
    for i in range(maxiter):
        XWt =  distmat.mm(data, W.t())
        WWt =  distmat.mm(W, W.t())
        VWWt = distmat.mm(V, WWt)
        V.mul_(XWt).div_(VWWt)

        VtX  = distmat.mm(V.t(), data, out_sizes=W.sizes)
        VtV  = distmat.mm(V.t(), V)
        VtVW = distmat.mm(VtV, W)
        W = W.mul_(VtX).div_(VtVW)
        if i % 10 == 0:
            # print obj
            pass

## $\ell_1$-regularized Cox Regression

In [ ]:
def cox_l1(rank, size):
    # TODO: initialization here
    lambd = 0.01
    soft_threshold = torch.nn.Softshrink(lambd)
    for i in range(maxiter):
        Xbeta = distmat.mm(data, beta)
        w = Xbeta.exp()
        W = w.cumsum(0)
        w_dist = distmat.dist_data(w, TType=w.TType)
        pi = (w_dist/W.t()) * pi_ind
        pd  = distmat.mm(pi, delta)
        dmpd = delta_dist - pd
        grad = distmat.mm(datat, dmpd)
        beta = (beta + grad * sigma).apply(soft_threshold)
        if i % 10 == 0:
            # print obj
            pass

## Multi-GPU Demonstration

We demonstrate 10,000 x 10,000 examples on 2-8 GPUs on our server.

## Multi-node

The data structure can also be utilized on multi-node clusters. The structure was used for the analysis of 200,000 x 500,000 UK Biobank data.

## Future Direction

MPI-only, lightweight, more flexible version in Julia is in preparation. CUDA-aware MPI support for the central MPI interface [MPI.jl](https://github.com/JuliaParallel/MPI.jl) was added in the process.